In [35]:
%config Completer.use_jedi = False

In [36]:
import pandas as pd

In [37]:
import cbpro
from auth_credentials import (api_secret,
                              api_key, 
                              api_pass)

In [38]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url           = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val   = msg.get('time',('-'*27))
            price_val  = msg.get('price',None)
            if price_val is not None:
                price_val = float(price_val)
            product_id = msg.get('product_id',None)
            
            print(f"{time_val:30} \
                {price_val:.3f} \
                {product_id}\tchannel type:{msg_type}")
    
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")

In [39]:
stream = TextWebsocketClient(products=['BTC-USD'],channels=['ticker'])
stream.start()

2021-07-11T06:14:31.685656Z                    33610.740                 BTC-USD	channel type:ticker
2021-07-11T06:14:36.678470Z                    33610.720                 BTC-USD	channel type:ticker


In [40]:
stream.close()

2021-07-11T06:14:41.692608Z                    33610.740                 BTC-USD	channel type:ticker
<---Websocket connection closed--->
	Total messages: 4


In [41]:
url='https://api-public.sandbox.pro.coinbase.com'

client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url=url
)

In [42]:
payment_methods = client.get_payment_methods()

In [43]:
for method in payment_methods:
    currency  = method.get('currency',None)
    if currency.upper() == 'USD':
        method_id = method.get('id',None) 
    elif currency is None:
        continue

print(f"Currency is `{currency}`\n")

Currency is `USD`



In [44]:
client.deposit(
    amount=5,
    currency=currency,
    payment_method_id=method_id
)

{'id': '8715ff2e-c3e3-4af3-a0f4-59d83c95ec2a',
 'amount': '5.00000000',
 'currency': 'USD',
 'payout_at': '2015-02-18T16:54:00-08:00'}

In [45]:
client.place_market_order(product_id='BTC-USD',side='buy',funds=500)

{'message': 'Insufficient funds'}

In [46]:
accounts = client.get_accounts()

In [47]:
for acc in accounts :
    currency = acc.get('currency')
    if currency=='BTC':
        acc_id = acc.get('id')

In [48]:
acc_history = client.get_account_history(acc_id)

In [49]:
import json 

for hist in acc_history:
    print(json.dumps(hist,indent=0))

In [59]:
# First import the libraries that we need to use
import pandas as pd
import requests
import json

def fetch_daily_data(symbol):
    pair_split = symbol.split('BTC/USD')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    #url = f'https://api.pro.coinbase.com/products/BTC/candles?granularity=60'
    url = 'https://api.pro.coinbase.com/products/BTC/USD/candles?start=2018-07-10T12:00:00&stop=2018-07-15T12:00:00&granularity=900'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)

    else:
        print("Did not receieve OK response from Coinbase API")


if __name__ == "__main__":
    # we set which pair we want to retrieve data for
    pair = "BTC/USD"
    fetch_daily_data(symbol=pair)

Did not receieve OK response from Coinbase API


In [61]:
client.get_product_ticker(product_id='BTC-USD')

{'trade_id': 31532245,
 'price': '33488.76',
 'size': '0.0002986',
 'time': '2021-07-11T06:36:56.082538Z',
 'bid': '33488.76',
 'ask': '33488.78',
 'volume': '10624.59840689'}

In [64]:
public_client = cbpro.PublicClient()

data = public_client.get_product_historic_rates('ETH-USD')


In [65]:
import cbpro
public_client = cbpro.PublicClient()

In [67]:
data = public_client.get_product_historic_rates('ETH-USD')

In [68]:
data

[[1625985960, 2092.65, 2093.74, 2092.69, 2093.66, 14.10479294],
 [1625985900, 2092.44, 2094.88, 2094.88, 2092.59, 68.63472029],
 [1625985840, 2094.89, 2096.34, 2095.94, 2094.89, 41.43659681],
 [1625985780, 2094.89, 2097.36, 2095.22, 2096.25, 92.73654219],
 [1625985720, 2094.9, 2097.2, 2097.2, 2095.41, 109.67344415],
 [1625985660, 2096.23, 2098.33, 2098.15, 2097.55, 51.45115495],
 [1625985600, 2096.86, 2098.62, 2097.37, 2098.56, 66.76250512],
 [1625985540, 2097.37, 2116.81, 2097.93, 2097.38, 967.69097752],
 [1625985480, 2093.42, 2107.62, 2093.85, 2097.61, 2632.14189645],
 [1625985420, 2089.01, 2099.11, 2089.35, 2093.97, 578.21547883],
 [1625985360, 2088.25, 2090.55, 2089.88, 2089.3, 53.43316245],
 [1625985300, 2088.77, 2092.53, 2092.53, 2089.33, 114.07982664],
 [1625985240, 2091.76, 2093.92, 2093.26, 2092.37, 84.86477801],
 [1625985180, 2093, 2096.47, 2095.78, 2093.27, 36.18071905],
 [1625985120, 2095.39, 2096.79, 2095.98, 2095.6, 24.96952375],
 [1625985060, 2095.35, 2098.9, 2098.02, 20

In [70]:
import pandas as pd
#import btalib	

In [77]:
data

[[1625985960, 2092.65, 2093.74, 2092.69, 2093.66],
 [1625985900, 2092.44, 2094.88, 2094.88, 2092.59],
 [1625985840, 2094.89, 2096.34, 2095.94, 2094.89],
 [1625985780, 2094.89, 2097.36, 2095.22, 2096.25],
 [1625985720, 2094.9, 2097.2, 2097.2, 2095.41],
 [1625985660, 2096.23, 2098.33, 2098.15, 2097.55],
 [1625985600, 2096.86, 2098.62, 2097.37, 2098.56],
 [1625985540, 2097.37, 2116.81, 2097.93, 2097.38],
 [1625985480, 2093.42, 2107.62, 2093.85, 2097.61],
 [1625985420, 2089.01, 2099.11, 2089.35, 2093.97],
 [1625985360, 2088.25, 2090.55, 2089.88, 2089.3],
 [1625985300, 2088.77, 2092.53, 2092.53, 2089.33],
 [1625985240, 2091.76, 2093.92, 2093.26, 2092.37],
 [1625985180, 2093, 2096.47, 2095.78, 2093.27],
 [1625985120, 2095.39, 2096.79, 2095.98, 2095.6],
 [1625985060, 2095.35, 2098.9, 2098.02, 2095.55],
 [1625985000, 2097.12, 2099.74, 2097.12, 2098.4],
 [1625984940, 2097.18, 2098.89, 2098.72, 2097.18],
 [1625984880, 2095.89, 2098.71, 2095.89, 2098.7],
 [1625984820, 2095.37, 2097.8, 2096.31, 20

In [72]:
for line in data:
    del line[5:]

In [76]:
data

[[1625985960, 2092.65, 2093.74, 2092.69, 2093.66],
 [1625985900, 2092.44, 2094.88, 2094.88, 2092.59],
 [1625985840, 2094.89, 2096.34, 2095.94, 2094.89],
 [1625985780, 2094.89, 2097.36, 2095.22, 2096.25],
 [1625985720, 2094.9, 2097.2, 2097.2, 2095.41],
 [1625985660, 2096.23, 2098.33, 2098.15, 2097.55],
 [1625985600, 2096.86, 2098.62, 2097.37, 2098.56],
 [1625985540, 2097.37, 2116.81, 2097.93, 2097.38],
 [1625985480, 2093.42, 2107.62, 2093.85, 2097.61],
 [1625985420, 2089.01, 2099.11, 2089.35, 2093.97],
 [1625985360, 2088.25, 2090.55, 2089.88, 2089.3],
 [1625985300, 2088.77, 2092.53, 2092.53, 2089.33],
 [1625985240, 2091.76, 2093.92, 2093.26, 2092.37],
 [1625985180, 2093, 2096.47, 2095.78, 2093.27],
 [1625985120, 2095.39, 2096.79, 2095.98, 2095.6],
 [1625985060, 2095.35, 2098.9, 2098.02, 2095.55],
 [1625985000, 2097.12, 2099.74, 2097.12, 2098.4],
 [1625984940, 2097.18, 2098.89, 2098.72, 2097.18],
 [1625984880, 2095.89, 2098.71, 2095.89, 2098.7],
 [1625984820, 2095.37, 2097.8, 2096.31, 20

In [73]:
dataframe = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
dataframe.set_index('date', inplace=True)
print(dataframe.head())

               open     high      low    close
date                                          
1625985960  2092.65  2093.74  2092.69  2093.66
1625985900  2092.44  2094.88  2094.88  2092.59
1625985840  2094.89  2096.34  2095.94  2094.89
1625985780  2094.89  2097.36  2095.22  2096.25
1625985720  2094.90  2097.20  2097.20  2095.41


In [75]:
dataframe.to_csv('HistoricRates.csv')